# Documentation Links

In [16]:
# How to decode text in NLG models: https://huggingface.co/blog/how-to-generate

# Pretrained models from documentation: https://huggingface.co/transformers/pretrained_models.html

# Pipeline API: https://huggingface.co/transformers/main_classes/pipelines.html#transformers.pipeline

# Model API: https://huggingface.co/transformers/main_classes/model.html

# Tokenizer API: https://huggingface.co/transformers/main_classes/tokenizer.html

# Configuration API: https://huggingface.co/transformers/main_classes/configuration.html

# Install Modules

In [17]:
!pip install -q transformers

# Import Modules

In [18]:
import nltk

from itertools import chain

from nltk.tokenize import RegexpTokenizer, sent_tokenize

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Download Dependencies

In [19]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Define Functions

In [20]:
def sentence_segmentation(document, minimum_n_words_to_accept_sentence, language):
    paragraphs = list(filter(lambda o: len(o.strip()) > 0, document.split('\n')))

    paragraphs = [ p.strip() for p in paragraphs ]

    paragraph_sentences = [ sent_tokenize(p, language=language) for p in paragraphs ]

    paragraph_sentences = chain(*paragraph_sentences)

    paragraph_sentences = [ s.strip() for s in paragraph_sentences ]

    normal_word_tokenizer = RegexpTokenizer(r'[^\W_]+')

    paragraph_sentences = filter(lambda o: len(normal_word_tokenizer.tokenize(o)) >= minimum_n_words_to_accept_sentence, paragraph_sentences)

    return list(paragraph_sentences)

def chunk_text_to_pieces(text, language, tokenizer):
	sentences = [ s + ' ' for s in sentence_segmentation(text, minimum_n_words_to_accept_sentence=1, language=language) ]

	chunks = []

	chunk = ''

	length = 0

	for sentence in sentences:
		tokenized_sentence = tokenizer.encode(sentence, truncation=False, max_length=None, return_tensors='pt') [0]

		if len(tokenized_sentence) > tokenizer.model_max_length:
			continue

		length += len(tokenized_sentence)

		if length <= tokenizer.model_max_length:
			chunk = chunk + sentence
		else:
			chunks.append(chunk.strip())
			chunk = sentence
			length = len(tokenized_sentence)

	if len(chunk) > 0:
		chunks.append(chunk.strip())

	return chunks

def clean_text(text):
  if text.count('.') == 0:
    return text.strip()

  end_index = text.rindex('.') + 1

  return text[0 : end_index].strip()

def summarize(text, model_key, device, language):
    tokenizer = AutoTokenizer.from_pretrained(model_key)

    model = AutoModelForSeq2SeqLM.from_pretrained(model_key)

    chunk_texts = chunk_text_to_pieces(text, language, tokenizer)

    chunk_summaries = []

    for chunk_text in chunk_texts:
      input_tokenized = tokenizer.encode(chunk_text, return_tensors='pt')

      input_tokenized = input_tokenized.to(device)

      summary_ids = model.to(device).generate(input_tokenized, length_penalty=3.0, min_length = int(0.1 * len(chunk_text)), max_length = int(0.2 * len(chunk_text)), early_stopping=True, num_beams=5, no_repeat_ngram_size=3)

      output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in summary_ids]

      chunk_summaries.append(output)

    summaries = [ clean_text(text) for chunk_summary in chunk_summaries for text in chunk_summary ]

    return summaries

# Models Selected (from Hugging Face)

In [21]:
# t5-base
# t5-small
# t5-large

# facebook/bart-large-cnn
# facebook/bart-large-xsum

# sshleifer/distilbart-cnn-12-6
# sshleifer/distilbart-cnn-6-6

# airKlizz/t5-base-multi-en-wiki-news
# airKlizz/t5-base-with-title-multi-en-wiki-news
# airKlizz/distilbart-multi-combine-wiki-news
# airKlizz/t5-small-multi-combine-wiki-news

# mrm8488/t5-base-finetuned-summarize-news

# Summarize a document (answers from Summer / Winter question)

In [22]:
document = '''
Ok, this is a tough dilemma for me. Winter is the season in which I mainly focus on my work and personal development. During this period I realize significant progress in all aspects of my life, from professional to personal level. On the other hand, summer is the time to finally relax and enjoy the results of all the hard work. So the most important factor for me to choose isn't the weather but the general focus in my day-to-day life.
Therefore, I can't decide on a single answer to this.
Life is good at every season and we have to make the most of it every day!
I love the sea, but not the extreme heat nor the mosquitoes, so it is an easy choice for me: winter! I like snow, and Christmas vacation, hot coffee and melomakarona! An ideal picture of a winter day: coffee in front of the fireplace with a book and good company.
When I was younger, I would definitely choose summer without a second thought. The feeling of being able to walk around freely due to the warm weather and sunlight makes Summer irresistible. Moreover, I related Summer to many enjoyable moments maybe because of the summer holiday. However, growing older I began to feel more satisfaction in cocooning at home while low temperatures and rain prevail outdoors. Besides, when it is cold you may very easily increase your body temperature by adding clothes whereas the inverse doesn't apply for warm weather. That's why I now choose winter.
Both are beautiful. However, I prefer summer more. There isn't a better feeling than relaxing on a beach at night and hearing the wave sounds. Summer is the time of ice creams and beautiful sunshines! Also, chilling out with friends, getting nothing done during the day except relaxing. In the summer the Sun brings energy to the whole planet and everything blooms again.
I prefer summer because it has the best weather. Winter can get boring due to bad weather and frequent rain. Also, I find summer the best time to have vacations.
Winter is cold, Summer is hot, the weather keeps changing from cold to hot and from hot to cold, you should like them all, the same way as you like to eat different kinds of food everyday. If the temperature stays hot or cold all year round, your life would be very boring. Some places on this earth have very little rain, dry and hot weather all year round like in the Middle East, many countries there have a few inches of rain the entire year, the temperature is hot all the time, but it’s very cold at night sometimes. People live in that parts of the planet have to like the weather they have to survive or they would die.
Well I believe it depends on the mood of the year. As a child I definitely preferred summer, as it was vacation season and schools were closed. On the other hand, when in university, I had a tendency to like winter more- summer had two exam periods, winter only one. Nowadays, both winter and summer are in the same preference level, but spring is my new favorite -warm afternoons and chill nights!
I definitely prefer summer, actually spring is even better exactly because summer is about to come. Spring gives us hope. Summer may or may let you down some times based on these expectations.
Can I have both? It is difficult to choose. Both have their beauties. However, if I was forced to choose, then it would be summer, as the warmer weather allows for more time outside. This together with the more time with light contribute to a better psychology and happiness. On the hedonistic side, summer with the beaches, the vacation, the more relaxed time is more free and sexy. At the end of the day it's summer hands down.
It’s an easy question for me, since I love summer and I cannot compare it with winter. Usually in winter I’m getting cold and it feels like I cannot get used to low temperatures. Additionally I prefer summer due to the fact that I love the sea and swimming. Summer vacations are the perfect time to leave back all the things that preoccupied you during winter (and usually there are a lot). Undoubtedly summer vacations give me the opportunity to hang out more with my friends and hearing Christina complain about mosquitoes. The good thing is that mosquitoes sting only Christina and let us unstingable :-p
I dislike cold weather, shoveling snow, paying for heating oil, fighting over Christmas bills, and having it get dark at 5:00 p.m., so the winter doesn’t have a great deal to offer me.
Summer, on the other hand (even though I, unfortunately, no longer enjoy a life situation where I can take the summers off) has a deep association with lazy warm evenings where I don’t have to be wearing a lot of clothes and I absolutely love that it stays light out longer. The light literally makes me more inclined to do stuff when I come home from work.
I love summer. I love riding my bike around in the heat. I love the profusion of flowers. I love our afternoon thunderstorms. I love the long days, the warm nights, maybe a vacation or a trip to the beach.
Most of all, I think, I love the idea of summer as a time off from our regular grind. Summer remains for me a liminal time, a period of transition between one year and the next. Something special happens in these periods of transition.
Winter for sure.
Typically people get mosquito bites and they have a little allergic reaction resulting in the mosquito bite. I get mosquito welts.
These things are monstrous and hurt to the point of no one can touch me if I have a couple. It sucks.
In the winter, there's no concern on mosquito bites and it's so beautiful. When everything is covered in that fresh blanket of snow, this will sound cheesy, but it makes everything look magical.
For me, winter. I hate summer. I feel uncomfortable. All I do is sweat all day long in the summer and that’s if I’m not doing anything physical. If I go outside to my shop to work on something, it’s even more miserable. All my hobbies are made worse by summer as well. Driving off road, no thanks. It’s not fun sitting in the machine feeling the engine heat soaking up through my seat.
So for me, I’ll take winter.
I much prefer the summer. The sun gives me energy. I do more and enjoy life more. The heat and humidity are only minor inconveniences. All I need is a shower and I am good to go.
In the winter. I never get warm once I am cold. I hate exercising because I don’t like to sweat and then be cold. I intensely dislike wearing the whole winter regalia when going out. Showers become difficult because we must keep the house cool because of heating expenses.
I love all the seasons, but after years of living in England I could do with a bit less winter and a bit more (decent) summer. Our autumns and springs are usually reasonable, while our summers and winters tend to be a wash of grey and rain that give me terrible migraines.
Real summer, the kind that comes with sunshine and blue skies and swimming in oceans and lakes, that I love. Real winter, the sort with snow and crisp starry skies and icy plumes of breath in the night air. I love that, too.
Summer of course. Why? Because it is warmer so I don’t have to put 50 layers of clothes on myself to feel warm, there is no need to wear long sleeved shirts and no need to use the heater to keep your room warm. I don’t get sick during it (unlike the winter when I’m sick every two weeks) and no eczema. And I was born in the summer so it’s one more thing to love it more than winter.
Of the two, I prefer winter. I love going for a brisk walk with the snow crunching under my boots and ending up at a favorite hangout for a meal, a Draught beer or a glass of wine. Or sitting by my wonderful gas fireplace reading, watching TV or listening to music.
I am more of a winter person. I grew up in the Midwest, where there is a sharp contrast between the two. I enjoy wearing warm clothing and the feeling of being indoors where it is toasty and warm. I also feel more lucid in cold weather. In summer it gets cold indoors due to air conditioning. I don’t like snow, though. I guess that to be exact, I am more of a late fall sort of person.
I love summers being immigrant from India. We do not like winter that much. I am cold even in summer. My body starts shivering when the temperature goes down due to rain and cold wind. I keep winter clothes handy all the year.
Only summer. Winter is miserable. Summer is the best time of the year. So much more fun.
I prefer the winter because I like wearing sweaters and lots of layers of clothing. In the summer it is too hot for that.
'''

In [23]:
model = 'sshleifer/distilbart-cnn-12-6'

device = 'cuda' # or 'cpu'

language = 'english'

summary_1 = ' '.join(summarize(document, model, device, language)) # First level summarization

print('\nFirst level summarization:\n')

print(summary_1)

summary_2 = ' '.join(summarize(summary_1, model, device, language)) # Second level summarization

print('\nSecond level summarization:\n')

print(summary_2)


First level summarization:

Winter is cold, Summer is hot, the weather keeps changing from cold to hot and from hot to cold, you should like them all, the same way as you like to eat different kinds of food everyday. Spring is my new favorite - warm afternoons and chill nights! I definitely prefer summer, actually spring is even better exactly because summer is about to come. Spring gives us hope. Summer may or may let you down some times based on expectations based on these expectations. Summer with the beaches, the vacation, the more relaxed time is more free and sexy. At the end of the day it's summer hands down. It’s an easy question for me, since I love summer and I cannot compare it with winter. Usually in winter I’m getting cold and it feels like I cannot get used to low temperatures. I like snow, and Christmas vacation, hot coffee and melomakarona. It is a good time to hang out with friends, getting nothing done during the day except relaxing. Undoubtedly summer vacations are 